In [1]:
from keras.layers import Input, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense, Dropout, Add
from keras.models import Model
from keras import backend as K

Using TensorFlow backend.


In [2]:
class ResidualBlock():
    ''' Residual Block
    '''
    
    def __init__(self, filters):
        
        self.filters = filters
    
    def __call__(self, x):
        
        x_short = x
        
        if x.shape[-1] != self.filters:
            x_short = Conv2D(filters=self.filters, kernel_size=(1, 1), strides=(2, 2), padding='same', activation='relu', kernel_initializer='he_normal')(x_short)
            x = Conv2D(filters=self.filters, kernel_size=(3, 3), strides=(2, 2), padding='same', activation='relu', kernel_initializer='he_normal')(x)
        else:
            x = Conv2D(filters=self.filters, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='he_normal')(x)
        
        x = Conv2D(filters=self.filters, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu')(x)
        
        x = Add()([x, x_short])

        return x

In [8]:
inputs = Input(shape=(224, 224, 3))

x = Conv2D(64, (7, 7), strides=(2, 2), padding='same', activation='relu')(inputs)
x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)

x = ResidualBlock(filters=64)(x)
x = ResidualBlock(filters=64)(x)
x = ResidualBlock(filters=64)(x)

x = ResidualBlock(filters=128)(x)
x = ResidualBlock(filters=128)(x)
x = ResidualBlock(filters=128)(x)
x = ResidualBlock(filters=128)(x)

x = ResidualBlock(filters=256)(x)
x = ResidualBlock(filters=256)(x)
x = ResidualBlock(filters=256)(x)
x = ResidualBlock(filters=256)(x)
x = ResidualBlock(filters=256)(x)
x = ResidualBlock(filters=256)(x)

x = ResidualBlock(filters=512)(x)
x = ResidualBlock(filters=512)(x)
x = ResidualBlock(filters=512)(x)

x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(1000, activation='softmax')(x)

resnet = Model(inputs=inputs, outputs=x)
resnet.compile(optimizer='sgd', loss='categorical_crossentropy')

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [9]:
resnet.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_6 (Conv2D)               (None, 112, 112, 64) 9472        input_6[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_5 (MaxPooling2D)  (None, 56, 56, 64)   0           conv2d_6[0][0]                   
__________________________________________________________________________________________________
conv2d_7 (Conv2D)               (None, 56, 56, 64)   36928       max_pooling2d_5[0][0]            
__________________________________________________________________________________________________
conv2d_8 (